# qBraid Quantum Labs: IBM Devices with your own credentials

Authors: Sophy Shin, James Weaver, Brian Ingmanson

This tutorial is about using Qiskit Runtime through qBraid Quantum Lab. The `Default` Environment supports most of the up-to-date versions of Qiskit, so the first step will be to bring your own credential from [IBM Quantum Platform](https://quantum.ibm.com/). 

If you do not already have a user account, get one at the [IBM Quantum login page](https://quantum.ibm.com/login). Your user account is associated with one or more [instances](https://docs.quantum.ibm.com/run/instances) (in the form hub / group / project) that give access to IBM Quantum services. Additionally, a unique token is assigned to each account, allowing for IBM Quantum access from Qiskit. The instructions in this section use our default instance. For instructions to choose a specific instance, see [Connect to an instance](https://docs.quantum.ibm.com/run/instances#connect-instance).

After logging in, at the top right, you can check the instances that you can use, and you can copy the API Token according to the instance at the right side of the banner by clicking the squared icon.

<img src="./img/ibm_api_token.png" />

If you are using `Open Plan` you can run your quantum circuits on IBM quantum systems for free (up to 10 minutes quantum time per month). See [IBM Quantum access plans](https://www.ibm.com/quantum/pricing) for details.

## 1. Using Qiskit Runtime Provider

Now you can start using IBM Quantum Backends by calling QiskitRuntimeService with your API Credential replace `<MY_IBM_QUANTUM_TOKEN>` by your own token:

In [ ]:
# set up service by using open plan instance. you can delete or modify to use another instance

from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService(
    channel="ibm_quantum", instance="ibm-q/open/main", token="<MY_IBM_QUANTUM_TOKEN>"
)

To view the backends you have access to, you can use the `QiskitRuntimeService.backends()` method. Let's check your backend list:

In [ ]:
service.backends()

The `QiskitRuntimeService.backend()` method (note that this is singular: backend) takes the name of the backend as the input parameter and returns an IBMBackend instance representing that particular backend. The following code will select `ibmq_qasm_simulator` and save it as a `backend_sim`

In [ ]:
backend_sim = service.backend("ibmq_qasm_simulator")

You can also filter the available backends by their properties. For more general filters, you can make advanced functions using a lambda function. Refer to the [API documentation](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/qiskit_ibm_runtime.QiskitRuntimeService#backends) for more details.

As shown here, we will filter least busy real backend and save it to `backend`.

In [ ]:
backend = service.least_busy(simulator=False, operational=True)
backend

### Create a toy circuit

Now, let's create a random circuit by using `qiskit.circuit.random.random_circuit` with 5 qubits with depth=3 with measurement. 

In [ ]:
from qiskit.circuit.random import random_circuit

circ = random_circuit(5, 3, measure=True)
circ.draw(output="mpl", style="iqp")

### Execute using a quantum primitive function

Quantum computers can produce random results, so you'll often want to collect a sample of the outputs by running the circuit many times. You can use the `Sampler` class to get measured data from a quantum Computer.. `Sampler` is one of our two [primitives](https://docs.quantum.ibm.com/run/primitives-get-started); the other is `Estimator`, which estimates the value of observable.

In [ ]:
from qiskit_ibm_runtime import Sampler, Options

options = Options()
options.resilience_level = 1
options.optimization_level = 3

# Create an Estimator object
sampler = Sampler(backend_sim, options=options)

# Submit the circuit to Estimator
job = sampler.run(circ, shots=10000)

You can print the job's id and status by using the job instance. Run below cell to check both.

In [ ]:
jobid = job.job_id()
print(f">>> Job ID: {job.job_id()}")
print(f">>> Job Status: {job.status()}")

<div class="alert alert-block alert-info">
<b>Note:</b> 
    Jobs submitted by using the qiskit_ibm_runtime provider are not visible in the right side panel of qBraid Lab </div>


### Retrieve job results at a later time

You can call service.job(\<job\how_toID>) to retrieve a job you previously submitted. If you don’t have the job ID, or if you want to retrieve multiple jobs at once; including jobs from retired systems, call service.jobs() with optional filters instead. See [QiskitRuntimeService.jobs](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/qiskit_ibm_runtime.QiskitRuntimeService#jobs) for details.

As shown here, we will retrieve the job result and save it as a `retrieve_job` to see the result.

In [ ]:
retrieve_job = service.job(jobid)
result = retrieve_job.result()

Now we will plot the results. 

As sampler returns quasi probability of measurement, let's use `plot_distribution` with a binary expression. See [SamplerResult document](https://docs.quantum.ibm.com/api/qiskit/qiskit.primitives.SamplerResult) for more information.

In [ ]:
from qiskit.visualization import plot_distribution

plot_distribution(result.quasi_dists[0].binary_probabilities())

## 2. Using qBraid SDK

By following [this lab demo](https://github.com/qBraid/qbraid-lab-demo/blob/045c7a8fbdcae66a7e64533dd9fe0e981dc02cf4/qbraid_sdk/ibm_batch_jobs_grovers.ipynb#L4) provided by qBraid, you can also use qbraid sdk to submit your ibm job and check job status. The following show how to do that.

First, check the qbraid version.

In [ ]:
import qbraid

qbraid.__version__

Now import essential libraries and save your ibm api token as an environment variable. Then, set-up your `QiskitProvider` with that token.

In [ ]:
from qbraid import device_wrapper, job_wrapper, get_jobs
from qbraid.providers import QuantumJob
from qbraid.providers.exceptions import JobStateError
from qbraid.providers.ibm import QiskitBackend, QiskitJob, QiskitProvider
import os

os.environ["QISKIT_IBM_TOKEN"] = "<MY_IBM_QUANTUM_TOKEN>"
ibmq_token = os.getenv("QISKIT_IBM_TOKEN")
provider = QiskitProvider(qiskit_ibm_token=ibmq_token)

You can also see the device list, accessible with your token.

In [ ]:
provider.get_devices()

`qbraid.providers.ibm` also supports useful filtering by `get_devices()`. You can quickly find the least busy backend by using `ibm_least_busy_gpu()`.

In [ ]:
# ibm_device = provider.ibm_least_busy_qpu() #return least busy backend of provider
# ibm_device = provider.get_devices(operational=True, simulator=False) #return backend list which is now operate and not a simulator
ibm_device = provider.get_device("ibm_kyoto")  # return backend by name

To send the quantum circuit to the backend and check the job status in the right sidebar of qBraid Quantum lab, wrap the ibm backend with `device_wrapper`. If you insert the backend, which does not appear in the right sidebar panel's "device" section, this code will return an error.

In [ ]:
# qbraid_ibm_device = device_wrapper(ibm_device) #you and add device called by provider as well
qbraid_ibm_device = device_wrapper("ibm_kyoto")

To send a quantum circuit, you can simply call `wrapped_device.run(circuit, options)`.  See [API document](https://docs.qbraid.com/en/stable/sdk/devices.html#device-wrapper) for more information.

In [ ]:
qbraid_ibm_job = qbraid_ibm_device.run(
    circ, shots=20000
)  # works if backend is at Devices list. Error if I try to use premium backend

Shortly afterwards you should see your submitted job in the right side panel. If you cannot see your job, click the circulation icon at the top to refresh or select `All` for Provider.

Also, you can check your job status by using `job_wrapper(job_id).` Please note that the `job_id` for `job_wrapper` must be a qBraidID, which you can get by adding `.id` to your job.

In [ ]:
job = job_wrapper(qbraid_ibm_job.id)
job.status()

You can use the `get_jobs` function to a return a list of your previously submitted quantum jobs, along with the status of each.

In [ ]:
get_jobs()

This `qBraidID` can be used to reinstantiate a qBraid QuantumJob object at any time, and even in a separate program, with no loss of information.

In [ ]:
job = job_wrapper("<MY_qBraid_Job_ID>")
job.status()

After the job has completed, we’ll gather the result, print the measurement counts, and plot a histogram of the count by using `qbraid.visualization.plot_histogram`

In [ ]:
ibm_result = job.result()

In [ ]:
from qbraid.visualization import plot_histogram, plot_distribution

plot_histogram(ibm_result.measurement_counts())

Also, you can plot a histogram of a probability with `plot_distribution`.

In [ ]:
plot_distribution(ibm_result.measurement_counts())